In [ ]:
from scipy import io
import pandas as pd
import umap # First time you run this enter pip install umap-learn in your Anaconda Prompt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, scale
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

In [ ]:
norm = 'E-GEOD-100911.aggregated_filtered_normalised_counts.mtx'
c = 'E-GEOD-100911.aggregated_filtered_normalised_counts.mtx_cols'
r = 'E-GEOD-100911.aggregated_filtered_normalised_counts.mtx_rows'
cols = []
rows = []
data = io.mmread(norm)
with open(c) as file:
    for line in file:
        cols.append(line.rstrip())

# with open(r) as file:
#     for line in file:
#         rows.append(line.rstrip().split('\t')[0])
# arr = data.toarray()

In [ ]:
# norm_counts = pd.DataFrame(arr, index = rows, columns = cols)
# norm_counts.info()

In [ ]:
# This was run to convert the gene transcript IDs to KEGG IDs, since this process took so long
# the result was saved to a csv (geneID.csv) file and this code is not needed anymore. 

# from Bio import Entrez
# import time
# import csv

# Entrez.email = "mdpouls1@gmail.com"
# geneIDs = []
# rowsTest = rows[15000:]
# i = 0
# while i < len(rowsTest):
#     for t in range(3):
#         if i < len(rowsTest):
#             handle = Entrez.esearch(db="gene", term=rowsTest[i])
#             record = Entrez.read(handle)
#             if len(record['IdList']) == 1:
#                 geneIDs.append(record['IdList'][0])
#             elif len(record['IdList']) > 1:
#                 geneIDs.append('multiple')
#             else:
#                 geneIDs.append('NaN')
#             handle.close()
#             i += 1
#     time.sleep(1)

# with open('geneID.csv', 'a') as f:
#     for gene in geneIDs:
#         f.write('{},'.format(gene))


In [ ]:
ID_Data = pd.read_csv('geneID.csv')
IDs = ID_Data.columns

In [ ]:
# norm_counts['geneIDs'] = IDs
# norm_counts.head(10)

In [ ]:
# with open('dataFile.txt', 'w') as file:
#     file.write(' '.join(norm_counts['geneIDs'].to_list()))

In [ ]:
annotationData = pd.read_table('uniprotData.tab')

In [ ]:
annotationData = annotationData.rename(columns={"yourlist:M20210331A94466D2655679D1FD8953E075198DA81B3BDFU": "Ids"})

In [ ]:
annotationData = annotationData.drop("yourlist:M20210331A94466D2655679D1FD8953E075198DA81B3B2BO", axis=1)

In [ ]:
annotationData.head()


In [ ]:
import pandas as pd
norm_counts = pd.read_csv('norm_counts_data.csv')


In [ ]:
norm_counts = norm_counts.set_index('Unnamed: 0')
norm_transpose = norm_counts.transpose()
norm_transpose.head()

In [ ]:
norm_transpose_dropped = norm_transpose.drop('geneIDs')
norm_transpose_dropped


In [ ]:
#geneID_dict['64604']

In [ ]:
# # #comvert the gene ontology codes to dummy columns in a dataframe for each gene ID then saved as a csv. 

# GO = []
# go_dict = {}
# for line in annotationData['Gene ontology (biological process)'].str.split(';'):
#     if type(line) == list:
#         for item in line:
#             GO.append(item.strip())

# for term in GO:
#     if term not in go_dict:
#         go_dict[term] = []

# for line in annotationData['Gene ontology (biological process)'].str.split(';'):
#     item_dict = {}
#     if type(line) == list:
#         for item in line:
#             item_dict[item.strip()] = True
#         for k,v in go_dict.items():
#             if k in item_dict:
#                 v.append(1)
#             else:
#                 v.append(0)

        
# go_df = pd.DataFrame.from_dict(go_dict)
# go_df['geneId'] = annotationData['Ids']

# go_df = go_df.set_index('geneId')
# go_df.head()

# go_df.to_csv('goData.csv')
    

In [ ]:
# for k,v in go_dict.items():
#     print(k)

In [ ]:
# norm_counts.head()

In [ ]:
# norm_counts_id = norm_counts.set_index("geneIDs")
# new = norm_counts.merge(go_df,left_on='geneIDs', right_on='geneId')
# new.head()

In [ ]:
# new = new.set_index('geneIDs')
# new.head()

In [ ]:
# srr_dict = norm_counts_id.to_dict()

In [ ]:
# srr_dict['SRR5810686']

In [ ]:
# gene_dict = go_df.to_dict('split')

In [ ]:
# geneID_dict = {}
# for ind, data in gene_dict.items():
#     geneID_dict[ind] = data

In [ ]:
# srr_dict = norm_counts_id.to_dict()
# gene_dict = go_df.to_dict('split')    

In [ ]:
# geneID_dict = {}
# for ind, data in gene_dict.items():
#     geneID_dict[ind] = data


In [ ]:
# ids = gene_dict['index']
# data = gene_dict['data']

# geneID_dict = {}
# for i in range(len(ids)):
#     geneID_dict[ids[i]] = data[i]

In [ ]:
# srr_genes_dict_values = {}
# for srr,genes in srr_dict.items():
#     genes_GO = np.zeros((5075,), dtype=int)
#     for gene,value in genes.items():
#         if value > 0:
#             if gene in geneID_dict:
#                 genes_GO = np.add(genes_GO,np.multiply(value, np.array(geneID_dict[gene])))
#     srr_genes_dict_values[srr] = genes_GO


In [ ]:
# cols = go_df.columns

In [ ]:
#gene_annotation_values_df = pd.DataFrame.from_dict(srr_genes_dict_values, orient='index', columns = cols)

In [ ]:
# gene_annotation_values_df.head()norm_counts['geneIDs'] = IDs
# norm_counts.head(10)

In [ ]:
# norm_counts.to_csv('norm_counts_data.csv')

In [ ]:
# with open('dataFile.txt', 'w') as file:
#     file.write(' '.join(norm_counts['geneIDs'].to_list()))

In [ ]:
#annotationData = pd.read_table('uniprotData.tab')

In [ ]:
#gene_annotation_values_df.to_csv('GeneAnnotation.csv')

In [ ]:
#annotationData = annotationData.drop("yourlist:M20210331A94466D2655679D1FD8953E075198DA81B3B2BO", axis=1)

In [ ]:
#annotationData.head()

# UMAP Reduction:

See the UMAP Documentation page for details. The module import cell at the top of this notebook has instructions for downloading the scikit learn plug-in for UMAP.

NOTE This is a first pass - I'm not super sure it is ready to go. Next steps

1 We need to cluster the data and assign tags to the data points

2 Add a third dimension to the UMAP reduction

3 Plot data in altair to allow interaction

4 Add interaction steps, tags, whatever else we're doing!


In [ ]:
norm_counts_temp = norm_counts
norm_counts_temp = norm_counts_temp.drop(['geneIDs'],axis=1).transpose()
norm_vals = norm_counts_temp.values

In [ ]:
norm_vals_scaled = StandardScaler().fit_transform(norm_vals)

In [ ]:
UMAP_reducer = umap.UMAP()
reduced_genes = UMAP_reducer.fit_transform(norm_vals_scaled)
print('Done')

In [ ]:
plt.scatter(reduced_genes[:,0],reduced_genes[:,1])
plt.gca().set_aspect('equal','datalim')
plt.title('UMAP projection of Zebra Fish genes, unclustered')
plt.show;

In [ ]:
X = norm_vals_scaled
# PCA
pca_mod = PCA(n_components = 7)
data_pca = pca_mod.fit_transform(X)
PCs =pca_mod.components_
PCs.shape

In [ ]:
UMAP_mod = umap.UMAP(n_neighbors = 7, min_dist = 0.2, n_components = 3).fit_transform(X)

In [ ]:
# K means
y_pred = KMeans(n_clusters=7, max_iter=5).fit_predict(X)

In [ ]:
# colors
from matplotlib.colors import ListedColormap
colors = ListedColormap(sns.color_palette('bright', 7).as_hex())
# plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(UMAP_mod[:, 0], UMAP_mod[:, 1],zs= UMAP_mod[:, 2], c=y_pred, cmap= colors, s=20)
ax.set_xlabel('Dimension 1')
ax.set_ylabel('Dimension 2')
ax.set_zlabel('Dimension 3')
plt.show()

In [ ]:
plt.scatter(UMAP_mod[:,0],UMAP_mod[:,1], c = y_pred, cmap = colors, s=15)
plt.gca().set_aspect('equal','datalim')
plt.title('UMAP projection of Zebra Fish genes, unclustered')
plt.show;

## Get top gene ontologies:  
**The** top gene ontologies are defined in this project as those that have the largest (absolute) parameter value in the final equation for the principal direction. 

In [ ]:
annotation = pd.read_csv('GeneAnnotation.csv')
annot_vals = annotation.values

annot_scaled = StandardScaler().fit_transform(annot_vals)


# PCA on gene ontologies: Use this to figure out het
pca_mod = PCA(n_components = 1) 
data_pca = pca_mod.fit_transform(annot_scaled)
PCs =pca_mod.components_

In [ ]:
PCs_df = pd.DataFrame(PCs)
PCs_df = PCs_df.transpose()
PCs_df[1] = abs(PCs_df[0])
topGo = PCs_df[1].sort_values(ascending = False).head(100).index #List of the index of the top gene ontologies

In [ ]:
top_annotations = annotation.iloc[:,topGo] # Grab the top 100 annotations
min_max_scaler = preprocessing.MinMaxScaler()
top_annotations[:] = min_max_scaler.fit_transform(top_annotations.values) #Scale the data to be in range [0,1]
top_annotations.columns = np.arange(0,len(top_annotations.columns))
top_annotations.head() #df with only the top 100 most important group ontologies listed for each cell

## Visualization:
**Overview**
 - The 'base' plot is the 2-D UMAP plot plotted using Altair. 
 - A user can then select a region-of-interest to create the breakout histogram plots of predominate gene ontology

In [ ]:
import altair as alt

### Create visualization dataframe:  
**Overview** by creating a dataframe with the reduced dimension data and add in additional properties as necessary (i.e. cluster number, ontology, etc.)  
  
**To-do:**  
 - Add cluster IDs
 - Add gene ontologies
 - Add primary genes?

In [ ]:
# Create 'base' dataframe:
column_names = ["UMAP x","UMAP y"]
vis_data = pd.DataFrame(reduced_genes, columns = column_names)
vis_data["Clusters"] = y_pred # Add clusters:
vis_data['IDs'] = top_annotations.index.values
# vis_data[top_annotations.columns] = top_annotations.values # Bring in the 'top annotations'

# vis_data.head()
# vis_data.plot.bar(y=1)
vis_data

In [ ]:
# Ceate a 'long' chart of the form [CellID | Annotation type | Annotation Value] (and probably UMAPs...)
annotation_play = top_annotations
annotation_play['ids'] = annotation_play.index
vals = np.arange(0,20)
annotation_play = pd.melt(top_annotations, id_vars=['ids'], value_vars=vals)
annotation_play['UMAP x'] = ''
annotation_play['UMAP y'] = ''
annotation_play['Cluster'] = ''

## ADD THE UMAP LOCATIONS AND CLUSTER ID FOR EACH INSTANCE OF EACH CELL:
# PS: Kinda hacked this one together.... so it takes a quick sec to run
cnt = 1
for i in range(len(annotation_play)):
    for j in range(len(vis_data)):
        if annotation_play.loc[i, 'ids'] == vis_data.loc[j,'IDs']:
            annotation_play.loc[i, 'UMAP x'] = float(vis_data.loc[j,'UMAP x'])
            annotation_play.loc[i, 'UMAP y'] = float(vis_data.loc[j,'UMAP y'])
            annotation_play.loc[i, 'Cluster'] = int(vis_data.loc[j,'Clusters'])
            print(len(annotation_play) - cnt)
            cnt += 1

annotation_play

### Create 'master' plot  
**TODO** 
 - Add cluster colors
 - Add IDs when hovering?
 - Add ROI selection


### Create histogram plot of ontologies:
**Thoughts:**
 - Altair does seem to be one of the easier plotting systems to select data
 - Altair does not output the indicies of the ROI without an API that may be out there...
 - I can 'transform' the data using:
  - Density Transform | create a vector of all of the ontologies, show a density plot of these ontologies
  - 

### Display tool!
**This is where our interaction is!**

In [ ]:
ROI = alt.selection_interval()

scatter_base = alt.Chart(annotation_play).mark_point().encode(
    x='UMAP x',
    y='UMAP y',
    color = 'Cluster'
).properties(
    width = 500,
    height = 200
).add_selection(ROI)

hist_base = alt.Chart(annotation_play).mark_bar().encode(
    x='variable',
    y='mean(value):Q'
).properties(
    width = 500,
    height = 200
).transform_filter(
    ROI
)

scatter_base & hist_base

## Main Dataset

 This is unpublished data from the Gagnon lab from the [cell ranger](https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/what-is-cell-ranger) pipeline. I loaded the files, cleaned the data and created a (large) csv file to load in.  As discussed in peer feedback, the unpublished data will be shared will a Google Drive link below.

In [ ]:
#testis1_data_ = io.mmread('matrix.mtx.gz')
#data_arr = testis1_data_.toarray()
#data_col = pd.read_csv('barcodes.tsv.gz', compression = 'gzip', header=None, sep = '\t')
#data_row = pd.read_csv('features.tsv.gz', compression = 'gzip', header=None,sep = '\t')
#data_rows = data_row[1]
#data_cols = []
#for b in data_col[0]:
#    data_cols.append(b)
#data_counts = pd.DataFrame(data_arr, index = data_rows, columns = data_cols)
#data_counts.to_csv('data_counts_data.csv')

### Here is the link to the csv file I generated with the code above: [t1_counts_data.csv](https://drive.google.com/file/d/1aIR4w9TIOnMxziE5aQVjIMAWvqEY6Mvq/view?usp=sharing)

In [ ]:
# The csv file above needs to be downloaded to load this cell
# It is a large dataset, so it may take a while to load
data_counts = pd.read_csv('t1_counts_data.csv', header = 0)
data_counts.set_index('1', inplace = True)
data_counts.shape

# Quality control of the cells

In [ ]:
# percentage of mitochondrial rna
mitochondrial = data_counts[data_counts.index.str.contains('mt-') > 0].sum(0)
total = data_counts.sum(0)
mito_percentage = (mitochondrial/total) * 100
filter_out = mito_percentage[mito_percentage < 5].index
data_counts = data_counts[filter_out] # filter out anything higher than 5%

# filter by counts of genes (a very large amount of transcribed genes implies more than one cell is captured)
gene_counts = []
for i,cell in enumerate(data_counts.columns):
    counter = 0
    for row in data_counts.iloc[:,i]:
        if row > 0:
            counter += 1
    gene_counts.append(counter)
count_mask = []
for count in gene_counts:
    if count > 200 & count < 2500:
        count_mask.append(gene_counts.index(count))
data_counts = data_counts.iloc[:,count_mask]
data_counts.shape

In [ ]:
# Define features (genes) and samples (cell_ids)
genes = data_counts.index.values
cell_ids = data_counts.columns